In [19]:
# Packages

import re
import pandas as pd
from Bio import Entrez, SeqIO, SeqFeature
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation

# Email for Entrez
Entrez.email ='your email'

In [20]:
# Open Twist order sheet as DataFrame

fasta_df = pd.read_excel('20240408_Twist_Order.xlsx')

# Remove FASTA formating of '>' character in 'Accession' column

fasta_df['Accession'] = fasta_df['Accession'].str[1:]

In [21]:
def downstream_sequence(row):
    
    '''
    This function takes Rfam entry information and outputs the coding gene downstream the entry.
    
    Input: Row from DataFrame FASTA file of 
        species' NCBI accession ID,
        genomic positions,
        Rfam entry sequence
    
    Output:
        codon_start = the location of the start position of the coding gene (provided by access_NCBI)
        gene = name of the coding gene (provided by access_NCBI)
    '''
    
    # Initialize values
    
    gene = int(0)
    x = 50 # Iterate over 50 nts looking for coding region
    
    # Rfam entry on the Positive strand:
    
    if row['Start'] < row['Stop']: 
        
        # Incimentially search further downstream the aptamer until hit coding region
        
        while type(gene) != str:
            
            # Establish positions in the genome
            
            start = row['Start']
            stop = row['Stop'] + x
            
            # Send genomic position to NCBI to identify downstream gene
            
            codon_start, gene = access_NCBI(start,stop, x) # gene returns a int(0) until coding region is found, which stops the loop
                
            # Check the next 50 nts for a protein retion    
                
            x = x + 50 
            
            if x > 1049: # Give up if no coding region is found within 1000 nts after the Rfam entry
                gene = 'Further than 1000 nts'
     
    # Rfam entry on the Negative strand:
    
    else:

        # Incimentially search further downstream the aptamer until hit protein
        
        while type(gene) != str :
            
            # Establish positions in the genome
            
            start = row['Stop'] - x
            stop = row['Start']
            
            # Send genomic position to NCBI to identify downstream gene
            
            codon_start, gene = access_NCBI(start,stop, x) # gene returns a int(0) until coding region is found, which stops the loop
                
            # Check the next 50 nts for a protein retion    
                
            x = x + 50
            
            if x > 1049: # Give up if no coding region is found within 1000 nts after the Rfam entry
                gene = 'Further than 1000 nts'
    
    return codon_start, gene

In [22]:
def access_NCBI(start, stop, x):
    """
    This function takes genomic positions and provides if there is a coding region and what it is.
    
    Input: 
        start = genome start position
        stop = genome stop position
        x = distance into the coding region to back track to start codon
    
    Output:
        codon_start = the location of the start position of the coding gene
        gene = name of the coding gene
    
    """
    
    # Collect genome region information from NCBI
            
    handle = Entrez.efetch(db="nucleotide", 
                           id=row['Accession'], 
                           rettype="gb", 
                           retmode="text", 
                           seq_start= start, 
                           seq_stop= stop)
            
    # Save genomic information

    GenomeSeq = SeqIO.read(handle, "genbank")

    for feature in GenomeSeq.features:

        # Identify CDS in genomic information
        if feature.type == "CDS":  
            location = feature.location
            if int(location.start) > 0:
                codon_start = int(location.start) - aptamer_length + 1
            else:
                codon_location = int(location.end)
                codon_start = int(x) - int(location.end)

            try:
                product = feature.qualifiers['product']
                gene = product[0]

            # If there are no CDS in the genomic information, the list is appended with 'unknown'

            except KeyError:
                gene = 'unknown'
    
            return codon_start, str(gene)
    
    return 'tbd' , int(0) # Loop continues until a coding region is found or x > 1049 nts from the entry

In [23]:
# Initalize lists

aptamer_length_list = []
strand_list = []
codon_start_list = []
aptamer_downstream_gene_list = []

# Calculate entry length

aptamer_length = len(row['Sequence'])

# Go through each DataFrame row

for index, row in fasta_df.iterrows():
    
    # Run functions to get coding information after Rfam entry
    
    aptamer_length, codon_start, aptamer_downstream_gene = downstream_sequence(row)
    
    # Save information into list
    
    aptamer_length_list.append(aptamer_length)
    codon_start_list.append(codon_start)
    aptamer_downstream_gene_list.append(aptamer_downstream_gene)

# Add infromation from lists to DataFrame

fasta_df['Aptamer Length'] = aptamer_length_list
fasta_df['Nts between Aptamer and Start Codon'] = codon_start_list
fasta_df['Gene'] = aptamer_downstream_gene_list

In [ ]:
# Export DataFrame to Excel

fasta_df.to_excel('Fluoride_CDS.xlsx', index=False)